## Adjusted return definitions

### Notations
Let $t_1$ be the ex-dividend date (or ex-date), $t_0$ be one day before ex-date; let $p_1, p_0$ be the close price on those two dates; let $d$ be the dividend amount.

### Yahoo
Yahoo claims that the adjustment is done "adhering to CRSP standards" [link](https://help.yahoo.com/kb/SLN28256.html) and shows  examples of how the "dividend multiplier" is computed as $$1 - \frac{d}{p_0}$$ Thus the return from $t_0$ to $t_1$ is $$\frac{p_1}{(1-\frac{d}{p_0})p_0}-1 = \frac{p_1}{p_0-d}-1 \qquad (1) $$

### CRSP
Per documentation [link](https://www.crsp.org/wp-content/uploads/guides/CRSP_US_Stock_&_Indexes_Database_Data_Descriptions_Guide.pdf) p. 86, "`Total Return = (adjprc + (divamt/cumfacpr/facpr))/pre_adjprc - 1`". For simplicity, assuming we are at the start of the price series, so that `adjprc` is just price; and there have been no splits, so that `cumfacpr` = `facpr` = 1, thus the return is computed as
$$ (p_1 + d)/ p_0 - 1 \qquad (2)$$

### Difference
A routine computation shows that
$$ (1) > (2) \quad \Longleftrightarrow \quad p_1-p_0+d > 0$$
Namely the sign of the error depends on if $p_1 \lessgtr p_0 - d$, namely whether the price just dropped by dividend amount on ex-date, which is the usual way to think about dividend payment effect on stock price.

In [1]:
import polars as pl
from polars import col as c
from datetime import datetime
import numpy as np
import pandas as pd

### Real world example 1

COSTCO issued a $15 dividend at year-end 2023 [link](https://investor.costco.com/stock-info/dividend-history/default.aspx)


In [2]:
df_costco_crsp = pl.read_parquet('/home/hzhang/data/wrds/crsp_costco_v2.parq')
df_costco_crsp = df_costco_crsp.filter((c('DlyCalDt').is_between(datetime(2023,12,26), datetime(2024,1,1), closed="both")))
with pl.Config(tbl_cols=-1):
    display(df_costco_crsp)

PERMNO,SecInfoStartDt,SecInfoEndDt,SecurityBegDt,SecurityEndDt,SecurityHdrFlg,HdrCUSIP,HdrCUSIP9,CUSIP,CUSIP9,PrimaryExch,ConditionalType,ExchangeTier,TradingStatusFlg,SecurityNm,ShareClass,USIncFlg,IssuerType,SecurityType,SecuritySubType,ShareType,SecurityActiveFlg,DelActionType,DelStatusType,DelReasonType,DelPaymentType,Ticker,TradingSymbol,PERMCO,SICCD,NAICS,ICBIndustry,NASDCompno,NASDIssuno,IssuerNm,YYYYMMDD,DlyCalDt,DlyDelFlg,DlyPrc,DlyPrcFlg,DlyCap,DlyCapFlg,DlyPrevPrc,DlyPrevPrcFlg,DlyPrevDt,DlyPrevCap,DlyPrevCapFlg,DlyRet,DlyRetx,DlyRetI,DlyRetMissFlg,DlyRetDurFlg,DlyOrdDivAmt,DlyNonOrdDivAmt,DlyFacPrc,DlyDistRetFlg,DlyVol,DlyClose,DlyLow,DlyHigh,DlyBid,DlyAsk,DlyOpen,DlyNumTrd,DlyMMCnt,DlyPrcVol,ShrStartDt,ShrEndDt,ShrOut,ShrSource,ShrFacType,ShrAdrFlg,DisExDt,DisSeqNbr,DisOrdinaryFlg,DisType,DisFreqType,DisPaymentType,DisDetailType,DisTaxType,DisOrigCurType,DisDivAmt,DisFacPr,DisFacShr,DisDeclareDt,DisRecordDt,DisPayDt,DisPERMNO,DisPERMCO,vwretd,vwretx,ewretd,ewretx,sprtrn
f64,datetime[ns],datetime[ns],datetime[ns],datetime[ns],str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str,str,f64,f64,str,f64,datetime[ns],str,f64,str,f64,str,f64,str,datetime[ns],f64,str,f64,f64,f64,str,str,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,datetime[ns],datetime[ns],f64,str,str,str,datetime[ns],f64,str,str,str,str,str,str,str,f64,f64,f64,datetime[ns],datetime[ns],datetime[ns],f64,f64,f64,f64,f64,f64,f64
87055.0,2023-10-01 00:00:00,2023-12-29 00:00:00,1985-11-27 00:00:00,2023-12-29 00:00:00,"""Y""","""22160K10""","""22160K105""","""22160K10""","""22160K105""","""Q""","""RW""","""S""","""A""","""COSTCO WHOLESALE CORP NEW; COM…","""""","""Y""","""CORP""","""EQTY""","""COM""","""NS""","""Y""","""N/A""","""UNAV""","""NACT""","""UNAV""","""COST""","""COST""",7882.0,5330.0,"""452319""","""NOAVAIL""",6.0012633e7,17254.0,"""COSTCO WHOLESALE CORP NEW""",2.0231226e7,2023-12-26 00:00:00,"""N""",674.62,"""TR""",2.9939e8,"""BP""",671.6,"""TR""",2023-12-22 00:00:00,2.9805e8,"""PB""",0.004497,0.004497,0.0,"""NA""","""D4""",0.0,0.0,1.0,"""NO""",1.760867e6,674.62,670.78,675.86,674.61,674.62,673.01,54823.0,55.0,1.1879e9,2023-11-24 00:00:00,2023-12-29 00:00:00,443787.0,"""OBS""","""NA""","""N""",null,null,"""""","""""","""""","""""","""""","""""","""""",null,null,null,null,null,null,null,null,0.005218,0.005199,0.009674,0.009299,0.004232
87055.0,2023-10-01 00:00:00,2023-12-29 00:00:00,1985-11-27 00:00:00,2023-12-29 00:00:00,"""Y""","""22160K10""","""22160K105""","""22160K10""","""22160K105""","""Q""","""RW""","""S""","""A""","""COSTCO WHOLESALE CORP NEW; COM…","""""","""Y""","""CORP""","""EQTY""","""COM""","""NS""","""Y""","""N/A""","""UNAV""","""NACT""","""UNAV""","""COST""","""COST""",7882.0,5330.0,"""452319""","""NOAVAIL""",6.0012633e7,17254.0,"""COSTCO WHOLESALE CORP NEW""",2.0231227e7,2023-12-27 00:00:00,"""N""",666.8,"""TR""",2.9592e8,"""BP""",674.62,"""TR""",2023-12-26 00:00:00,2.9939e8,"""PB""",0.010643,-0.011592,0.022235,"""NA""","""D1""",15.0,0.0,1.0,"""C1""",1.855047e6,666.8,660.4,668.58,666.82,667.0,662.7,63649.0,55.0,1.2369e9,2023-11-24 00:00:00,2023-12-29 00:00:00,443787.0,"""OBS""","""NA""","""N""",2023-12-27 00:00:00,1.0,"""Y""","""SD""","""E""","""USD""","""SDIV""","""D""","""USD""",15.0,0.0,0.0,2023-12-14 00:00:00,2023-12-28 00:00:00,2024-01-12 00:00:00,0.0,0.0,0.001995,0.001807,0.005276,0.004907,0.00143
87055.0,2023-10-01 00:00:00,2023-12-29 00:00:00,1985-11-27 00:00:00,2023-12-29 00:00:00,"""Y""","""22160K10""","""22160K105""","""22160K10""","""22160K105""","""Q""","""RW""","""S""","""A""","""COSTCO WHOLESALE CORP NEW; COM…","""""","""Y""","""CORP""","""EQTY""","""COM""","""NS""","""Y""","""N/A""","""UNAV""","""NACT""","""UNAV""","""COST""","""COST""",7882.0,5330.0,"""452319""","""NOAVAIL""",6.0012633e7,17254.0,"""COSTCO WHOLESALE CORP NEW""",2.0231228e7,2023-12-28 00:00:00,"""N""",663.1,"""TR""",2.9428e8,"""BP""",666.8,"""TR""",2023-12-27 00:00:00,2.9592e8,"""PB""",-0.0055

In [3]:
df = df_costco_crsp
np.testing.assert_almost_equal( df[1, 'DlyRet'] , 
                               (df[1, 'DlyClose'] + df[1, 'DlyOrdDivAmt'])/df[0, 'DlyClose'] -1)
np.testing.assert_almost_equal( df[1, 'DlyRetx'] , (df[1, 'DlyClose'] )/df[0, 'DlyClose'] -1)
np.testing.assert_almost_equal( df[1, 'DlyRetI'] , (df[1, 'DlyOrdDivAmt'])/df[0, 'DlyClose'])

In [4]:
# price and dividend: https://finance.yahoo.com/quote/COST/history/?period1=1580417983&period2=1738270775
df_costco_yahoo =pl.DataFrame({'date': [datetime(2023,12,26), datetime(2023,12,27), datetime(2023,12,28), datetime(2023,12,29)],
                               'adj_close': [655.79, 662.93, 659.25,656.25],
                               'close': [674.62, 668.80, 663.10, 660.08]
                              })
df_costco_yahoo

date,adj_close,close
datetime[μs],f64,f64
2023-12-26 00:00:00,655.79,674.62
2023-12-27 00:00:00,662.93,668.8
2023-12-28 00:00:00,659.25,663.1
2023-12-29 00:00:00,656.25,660.08


In [5]:
df = df_costco_yahoo
df_costco_yahoo_t0_adjprice = (df[1,'adj_close'] / df[1,'close']  # t_1 adjustment factor
                       * (1 - 15.0/df[0,'close']) # dividend adjustment factor per Yahoo definition
                       * df[0, 'close'])
df_costco_yahoo_t0_adjprice

653.830572069378

### Real world example 2

CVI in mid Aug 2023 [link](https://cvrenergy.gcs-web.com/cash-dividends)

but I cannot really reproduce Yahoo's computation (why do I match only to 2 sig fig's, rather than 4 sig fig's considering rounding?).

⏳Might have to wait another data source for comparison.

In [6]:
# price and dividend: https://finance.yahoo.com/quote/CVI/history/?period1=1690934400&period2=1692662400
df_cvi_yahoo =pl.DataFrame({'date': [datetime(2023,8,10), datetime(2023,8,11)],
                               'adj_close': [31.97, 32.49],
                               'close': [36.55, 35.99]
                              })

In [7]:
df = df_cvi_yahoo
df_cvi_yahoo_t0_adjprice = (df[1,'adj_close'] / df[1,'close']  # t_1 adjustment factor
                       * (1 - 1.5/df[0,'close']) # dividend adjustment factor per Yahoo definition
                       * df[0, 'close'])
df_cvi_yahoo_t0_adjprice

31.641414281744925

In [8]:
df_cvi_crsp = pl.read_parquet('/home/hzhang/data/wrds/crsp_cvi_v2.parq')
df_cvi_crsp = df_cvi_crsp.filter(c('DlyCalDt').is_between(datetime(2023,8,10), datetime(2023,8,11), closed="both"))
df_cvi_crsp

PERMNO,SecInfoStartDt,SecInfoEndDt,SecurityBegDt,SecurityEndDt,SecurityHdrFlg,HdrCUSIP,HdrCUSIP9,CUSIP,CUSIP9,PrimaryExch,ConditionalType,ExchangeTier,TradingStatusFlg,SecurityNm,ShareClass,USIncFlg,IssuerType,SecurityType,SecuritySubType,ShareType,SecurityActiveFlg,DelActionType,DelStatusType,DelReasonType,DelPaymentType,Ticker,TradingSymbol,PERMCO,SICCD,NAICS,ICBIndustry,NASDCompno,NASDIssuno,IssuerNm,YYYYMMDD,DlyCalDt,…,DlyClose,DlyLow,DlyHigh,DlyBid,DlyAsk,DlyOpen,DlyNumTrd,DlyMMCnt,DlyPrcVol,ShrStartDt,ShrEndDt,ShrOut,ShrSource,ShrFacType,ShrAdrFlg,DisExDt,DisSeqNbr,DisOrdinaryFlg,DisType,DisFreqType,DisPaymentType,DisDetailType,DisTaxType,DisOrigCurType,DisDivAmt,DisFacPr,DisFacShr,DisDeclareDt,DisRecordDt,DisPayDt,DisPERMNO,DisPERMCO,vwretd,vwretx,ewretd,ewretx,sprtrn
f64,datetime[ns],datetime[ns],datetime[ns],datetime[ns],str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str,str,f64,f64,str,f64,datetime[ns],…,f64,f64,f64,f64,f64,f64,f64,f64,f64,datetime[ns],datetime[ns],f64,str,str,str,datetime[ns],f64,str,str,str,str,str,str,str,f64,f64,f64,datetime[ns],datetime[ns],datetime[ns],f64,f64,f64,f64,f64,f64,f64
92326.0,2008-06-27 00:00:00,2023-09-28 00:00:00,2007-10-23 00:00:00,2023-12-29 00:00:00,"""N""","""12662P10""","""12662P108""","""12662P10""","""12662P108""","""N""","""RW""","""N/A""","""A""","""C V R ENERGY; COM NONE; CONS""","""""","""Y""","""CORP""","""EQTY""","""COM""","""NS""","""Y""","""N/A""","""UNAV""","""NACT""","""UNAV""","""CVI""","""CVI""",52793.0,2911.0,"""324110""","""ENERGY""",0.0,0.0,"""C V R ENERGY""",2.023081e7,2023-08-10 00:00:00,…,37.46,37.08,38.63,37.45,37.46,37.67,null,null,4.4605e7,2023-07-31 00:00:00,2023-08-30 00:00:00,100531.0,"""OBS""","""NA""","""N""",null,null,"""""","""""","""""","""""","""""","""""","""""",null,null,null,null,null,null,null,null,0.000093,-0.000024,-0.002073,-0.002202,0.000251
92326.0,2008-06-27 00:00:00,2023-09-28 00:00:00,2007-10-23 00:00:00,2023-12-29 00:00:00,"""N""","""12662P10""","""12662P108""","""12662P10""","""12662P108""","""N""","""RW""","""N/A""","""A""","""C V R ENERGY; COM NONE; CONS""","""""","""Y""","""CORP""","""EQTY""","""COM""","""NS""","""Y""","""N/A""","""UNAV""","""NACT""","""UNAV""","""CVI""","""CVI""",52793.0,2911.0,"""324110""","""ENERGY""",0.0,0.0,"""C V R ENERGY""",2.0230811e7,2023-08-11 00:00:00,…,36.55,35.79,37.07,36.55,36.56,35.89,null,null,3.0012e7,2023-07-31 00:00:00,2023-08-30 00:00:00,100531.0,"""OBS""","""NA""","""N""",2023-08-11 00:00:00,1.0,"""Y""","""CD""","""Q""","""USD""","""CDIV""","""D""","""USD""",0.5,0.0,0.0,2023-07-31 00:00:00,2023-08-14 00:00:00,2023-08-21 00:00:00,0.0,0.0,-0.000907,-0.001037,-0.001429,-0.001489,-0.00107
92326.0,2008-06-27 00:00:00,2023-09-28 00:00:00,2007-10-23 00:00:00,2023-12-29 00:00:00,"""N""","""12662P10""","""12662P108""","""12662P10""","""12662P108""","""N""","""RW""","""N/A""","""A""","""C V R ENERGY; COM NONE; CONS""","""""","""Y""","""CORP""","""EQTY""","""COM""","""NS""","""Y""","""N/A""","""UNAV""","""NACT""","""UNAV""","""CVI""","""CVI""",52793.0,2911.0,"""324110""","""ENERGY""",0.0,0.0,"""C V R ENERGY""",2.0230811e7,2023-08-11 00:00:00,…,36.55,35.79,37.07,36.55,36.56,35.89,null,null,3.0012e7,2023-07-31 00:00:00,2023-08-30 00:00:00,100531.0,"""OBS""","""NA""","""N""",2023-08-11 00:00:00,2.0,"""Y""","""SD""","""E""","""USD""","""SDIV""","""D""","""USD""",1.0,0.0,0.0,2023-07-31 00:00:00,2023-08-14 00:00:00,2023-08-21 00:00:00,0.0,0.0,-0.000907,-0.001037,-0.001429,-0.001489,-0.00107


In [9]:
df = df_cvi_crsp
np.testing.assert_almost_equal( df[1, 'DlyRet'] , 
                               (df[1, 'DlyClose'] + df[1, 'DlyOrdDivAmt'])/df[0, 'DlyClose'] -1)
np.testing.assert_almost_equal( df[1, 'DlyRetx'] , (df[1, 'DlyClose'] )/df[0, 'DlyClose'] -1)
np.testing.assert_almost_equal( df[1, 'DlyRetI'] , (df[1, 'DlyOrdDivAmt'])/df[0, 'DlyClose'])

In [10]:
raise InterruptedError

InterruptedError: 

In [ ]:
fn = 'crsp_2023_v2'
df = pd.read_stata(f'/home/hzhang/data/wrds/{fn}.dta.gz')
df_crsp  = pl.from_pandas(df)
df_crsp.write_parquet(f'/home/hzhang/data/wrds/{fn}.parq')

In [ ]:
df = pl.read_parquet('/home/hzhang/data/wrds/crsp_2023_v2.parq')

In [ ]:
with pl.Config(tbl_cols=-1):
    display(df.filter(c('DlyNonOrdDivAmt').ne_missing(0))[:4])